## code for generating ATC codes from rxcui and NDC
ultimate goal: We want a list of all antibiotics prescribed in the prescription table that would be prescribed if the physician suspected the patient of having a serious bacterial infection


* ATC codes are obtained via two step conversion:
 * national drug codes (NDC) found in the PRESCRIPTION table of the MIMIC-III dataset -> RxNorm concept unique identifier (RXCUI) codes. 
 * -> RXCUI codes into ATC codes.

* ATC codes with prefix 'J01' are considered to be antibiotics. 
* Regular expressions were used on prescription names to further filter out erroneous entries and entries with missing NDC/RXCUI codes. 
 * note: the text analysis via manual review and regular expressions is located in __. in this notebook, all erroneous values are hardcoded in a list to filter out.

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime


time: 0 ns (started: 2023-07-20 09:50:55 +08:00)


## Fetch all responses within one Client session first

In [2]:
import asyncio
from aiohttp import ClientSession
import time
import json

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.read()

async def run(url_lst):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    start = time.perf_counter()
    
    async with ClientSession() as session:
        for url in url_lst:
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
#         print(responses)

    duration = time.perf_counter() - start
    print('total time: {}'.format(duration))
    return responses

time: 203 ms (started: 2023-07-20 09:50:55 +08:00)


## Access MIMIC database and convert it to dataframe in Pandas

In [3]:
# note, all server information is stored in a config.py file that is present in the .gitignore
import config 
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciv" + ';'

time: 47 ms (started: 2023-07-20 09:50:55 +08:00)


## Prescription
* grabbing the entire prescription table from MIMIC

In [4]:
#code for generating ATC codes from rxcui and NDC

time: 0 ns (started: 2023-07-20 09:50:55 +08:00)


In [5]:
cur.execute('Select * from mimiciv_hosp.prescriptions')
rows = cur.fetchall()

time: 1min 16s (started: 2023-07-20 09:50:55 +08:00)


In [6]:
col_name = []
for i in range(len(cur.description)):
    col_name.append(cur.description[i][0])
print (col_name)

['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq', 'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug', 'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx', 'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route']
time: 16 ms (started: 2023-07-20 09:52:12 +08:00)


In [7]:
prescriptionTable = pd.DataFrame(rows,columns=col_name)

time: 1min 26s (started: 2023-07-20 09:52:12 +08:00)


In [8]:
prescriptionTable.head(10)

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10418005,22442981,61774509,10418005-61,61.0,P8558I,2130-10-14 17:00:00,2130-10-15 16:00:00,MAIN,Potassium Chloride,...,015362,00338070948,10mEq/100mL Premix,,40,mEq,400,mL,1.0,IV
1,10418005,22442981,61993757,10418005-75,75.0,P8558I,2130-10-14 20:00:00,2130-10-15 19:00:00,MAIN,Heparin,...,006549,63323026201,5000 Units / mL- 1mL Vial,None,5000,UNIT,1,mL,2.0,SC
2,10418005,22442981,73394102,10418005-37,37.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Influenza Vaccine Quadrivalent,...,075715,49281041688,0.5 mL Syringe,None,0.5,mL,1,SYR,0.0,IM
3,10418005,22442981,77280491,10418005-86,86.0,P8558I,2130-10-15 09:00:00,2130-10-16 18:00:00,MAIN,Ibuprofen,...,008349,00904585461,600mg Tablet,None,600,mg,1,TAB,NaN,PO
4,10418005,22442981,83578254,10418005-89,89.0,P8558I,2130-10-15 11:00:00,2130-10-16 18:00:00,MAIN,Potassium Chloride,...,001262,00603155404,20mEq Packet,None,60,mEq,3,PKT,1.0,PO
5,10418005,22442981,87576600,10418005-76,76.0,P8558I,2130-10-15 08:00:00,2130-10-14 17:00:00,MAIN,Enoxaparin Sodium,...,039482,63323056887,40mg Syringe,None,40,mg,1,SYR,1.0,SC
6,10418005,22442981,88929421,10418005-47,47.0,P59AIO,2130-10-13 19:00:00,2130-10-14 18:00:00,MAIN,Ondansetron,...,061716,00641607825,2mg/mL-2mL,None,4,mg,1,VIAL,NaN,IV
7,10418005,22442981,91062360,10418005-72,72.0,P11TUA,2130-10-14 13:00:00,2130-10-16 18:00:00,MAIN,OxyCODONE (Immediate Release),...,004225,00904644461,5mg Tablet,None,5,mg,1,TAB,NaN,PO
8,10418005,22442981,91535698,10418005-45,45.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Sodium Chloride 0.9% Flush,...,None,0,10 mL Syringe,None,3-10,mL,0.3-1,SYR,3.0,IV
9,10418005,22442981,94073691,10418005-66,66.0,P27H2N,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Glucose Gel,...,001781,38396055018,15 g Tube,None,15,g,1,TUBE,NaN,PO


time: 297 ms (started: 2023-07-20 09:53:39 +08:00)


## Convert NDC to Rxcui and add it in to prescription table
* must ping nlm url to do this. 

In [9]:
import nest_asyncio
nest_asyncio.apply()

time: 31 ms (started: 2023-07-20 09:53:40 +08:00)


In [10]:
ndc_lst = prescriptionTable['ndc'].unique()
url_lst = [#'http://rxnav.nlm.nih.gov/REST/rxcui.json?idtype=NDC&id={0}'.format(ndc)
           'https://rxnav.nlm.nih.gov/REST/ndcstatus.json?ndc={0}'.format(ndc)
           for ndc in ndc_lst]

loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(url_lst))
responses = loop.run_until_complete(future)

total time: 18.105565599999863
time: 20.7 s (started: 2023-07-20 09:53:40 +08:00)


In [11]:
#rxcui
def get_rxnormId_by_ndc(response):
    try:
        #class_lst = [x['rxclassMinConceptItem']['classId'] for x in json.loads(response)['rxclassDrugInfoList']['rxclassDrugInfo']]
        return json.loads(response)['ndcStatus']['rxcui']
    except:
        return ''

time: 0 ns (started: 2023-07-20 09:54:00 +08:00)


In [12]:
Rx_code_lst = [get_rxnormId_by_ndc(resp) for resp in responses] #list of rxnorm_codes

#making a dictionary of mappings for ndc_code: rxcui
ndc_rc_code_lst_map = {ndc: rx for ndc, rx in zip(ndc_lst, Rx_code_lst)}


time: 62 ms (started: 2023-07-20 09:54:00 +08:00)


In [13]:
##make a column that mapps the NDC code to RXCUI
prescriptionTable = prescriptionTable.assign(rxcui = prescriptionTable['ndc'].map(ndc_rc_code_lst_map))

time: 5.94 s (started: 2023-07-20 09:54:00 +08:00)


In [14]:
prescriptionTable.head(10)

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,rxcui
0,10418005,22442981,61774509,10418005-61,61.0,P8558I,2130-10-14 17:00:00,2130-10-15 16:00:00,MAIN,Potassium Chloride,...,00338070948,10mEq/100mL Premix,,40,mEq,400,mL,1.0,IV,312507
1,10418005,22442981,61993757,10418005-75,75.0,P8558I,2130-10-14 20:00:00,2130-10-15 19:00:00,MAIN,Heparin,...,63323026201,5000 Units / mL- 1mL Vial,None,5000,UNIT,1,mL,2.0,SC,1361615
2,10418005,22442981,73394102,10418005-37,37.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Influenza Vaccine Quadrivalent,...,49281041688,0.5 mL Syringe,None,0.5,mL,1,SYR,0.0,IM,1801186
3,10418005,22442981,77280491,10418005-86,86.0,P8558I,2130-10-15 09:00:00,2130-10-16 18:00:00,MAIN,Ibuprofen,...,00904585461,600mg Tablet,None,600,mg,1,TAB,NaN,PO,197806
4,10418005,22442981,83578254,10418005-89,89.0,P8558I,2130-10-15 11:00:00,2130-10-16 18:00:00,MAIN,Potassium Chloride,...,00603155404,20mEq Packet,None,60,mEq,3,PKT,1.0,PO,1867544
5,10418005,22442981,87576600,10418005-76,76.0,P8558I,2130-10-15 08:00:00,2130-10-14 17:00:00,MAIN,Enoxaparin Sodium,...,63323056887,40mg Syringe,None,40,mg,1,SYR,1.0,SC,854235
6,10418005,22442981,88929421,10418005-47,47.0,P59AIO,2130-10-13 19:00:00,2130-10-14 18:00:00,MAIN,Ondansetron,...,00641607825,2mg/mL-2mL,None,4,mg,1,VIAL,NaN,IV,1740467
7,10418005,22442981,91062360,10418005-72,72.0,P11TUA,2130-10-14 13:00:00,2130-10-16 18:00:00,MAIN,OxyCODONE (Immediate Release),...,00904644461,5mg Tablet,None,5,mg,1,TAB,NaN,PO,1049621
8,10418005,22442981,91535698,10418005-45,45.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Sodium Chloride 0.9% Flush,...,0,10 mL Syringe,None,3-10,mL,0.3-1,SYR,3.0,IV,
9,10418005,22442981,94073691,10418005-66,66.0,P27H2N,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Glucose Gel,...,38396055018,15 g Tube,None,15,g,1,TUBE,NaN,PO,198683


time: 31 ms (started: 2023-07-20 09:54:06 +08:00)


## Convert Rxcui to ATC and check whether the drug is Anibiotics, and add it into table

In [15]:
rxcui_lst = prescriptionTable['rxcui'].unique()
url_lst = ['https://rxnav.nlm.nih.gov/REST/rxclass/class/byRxcui.json?rxcui={}&relaSource=ATC'.format(rxcui)
           for rxcui in rxcui_lst]

loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(url_lst))
responses = loop.run_until_complete(future)

total time: 9.572629600000027
time: 10.3 s (started: 2023-07-20 09:54:06 +08:00)


In [16]:
def get_ATC_code_lst(response):
    try:
        class_lst = [x['rxclassMinConceptItem']['classId'] for x in json.loads(response)['rxclassDrugInfoList']['rxclassDrugInfo']]
        return class_lst
    except:
        return []

ATC_code_lst_lst = [get_ATC_code_lst(resp) for resp in responses]

#qc to check it works
pd.DataFrame({'rxcui': rxcui_lst,
                   'ATC_code_lst': ATC_code_lst_lst}).head(5)

,rxcui,ATC_code_lst
0,312507,"[A12BA, B05XA]"
1,1361615,"[B01AB, C05BA, S01XA]"
2,1801186,[]
3,197806,"[C01EB, G02CC, M01AE, M02AA, R02AX]"
4,1867544,"[A12BA, B05XA]"


time: 31 ms (started: 2023-07-20 09:54:17 +08:00)


In [17]:
ab_lst = [any(map(lambda x: x.startswith('J01'), ATC_code_lst)) 
           for ATC_code_lst in ATC_code_lst_lst]

#qc to check it works
pd.DataFrame({'rxcui': rxcui_lst,
                   'ATC_code_lst': ATC_code_lst_lst,
                   'ab': ab_lst}).head(35)

,rxcui,ATC_code_lst,ab
0,312507,"[A12BA, B05XA]",False
1,1361615,"[B01AB, C05BA, S01XA]",False
2,1801186,[],False
3,197806,"[C01EB, G02CC, M01AE, M02AA, R02AX]",False
4,1867544,"[A12BA, B05XA]",False
5,854235,[B01AB],False
6,1740467,[A04AA],False
7,1049621,[N02AA],False
8,,[],False
9,198683,"[B05CX, V04CA, V06DC]",False


time: 31 ms (started: 2023-07-20 09:54:17 +08:00)


In [23]:
prescriptionTable.head(10)

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,rxcui,Antibiotics
0,10418005,22442981,61774509,10418005-61,61.0,P8558I,2130-10-14 17:00:00,2130-10-15 16:00:00,MAIN,Potassium Chloride,...,10mEq/100mL Premix,,40,mEq,400,mL,1.0,IV,312507,False
1,10418005,22442981,61993757,10418005-75,75.0,P8558I,2130-10-14 20:00:00,2130-10-15 19:00:00,MAIN,Heparin,...,5000 Units / mL- 1mL Vial,None,5000,UNIT,1,mL,2.0,SC,1361615,False
2,10418005,22442981,73394102,10418005-37,37.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Influenza Vaccine Quadrivalent,...,0.5 mL Syringe,None,0.5,mL,1,SYR,0.0,IM,1801186,False
3,10418005,22442981,77280491,10418005-86,86.0,P8558I,2130-10-15 09:00:00,2130-10-16 18:00:00,MAIN,Ibuprofen,...,600mg Tablet,None,600,mg,1,TAB,NaN,PO,197806,False
4,10418005,22442981,83578254,10418005-89,89.0,P8558I,2130-10-15 11:00:00,2130-10-16 18:00:00,MAIN,Potassium Chloride,...,20mEq Packet,None,60,mEq,3,PKT,1.0,PO,1867544,False
5,10418005,22442981,87576600,10418005-76,76.0,P8558I,2130-10-15 08:00:00,2130-10-14 17:00:00,MAIN,Enoxaparin Sodium,...,40mg Syringe,None,40,mg,1,SYR,1.0,SC,854235,False
6,10418005,22442981,88929421,10418005-47,47.0,P59AIO,2130-10-13 19:00:00,2130-10-14 18:00:00,MAIN,Ondansetron,...,2mg/mL-2mL,None,4,mg,1,VIAL,NaN,IV,1740467,False
7,10418005,22442981,91062360,10418005-72,72.0,P11TUA,2130-10-14 13:00:00,2130-10-16 18:00:00,MAIN,OxyCODONE (Immediate Release),...,5mg Tablet,None,5,mg,1,TAB,NaN,PO,1049621,False
8,10418005,22442981,91535698,10418005-45,45.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Sodium Chloride 0.9% Flush,...,10 mL Syringe,None,3-10,mL,0.3-1,SYR,3.0,IV,,False
9,10418005,22442981,94073691,10418005-66,66.0,P27H2N,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Glucose Gel,...,15 g Tube,None,15,g,1,TUBE,NaN,PO,198683,False


time: 16 ms (started: 2023-07-20 10:26:32 +08:00)


In [18]:
ab_map = {rxcui: ab for rxcui, ab in zip(rxcui_lst, ab_lst)}

prescriptionTable = prescriptionTable.assign(Antibiotics = prescriptionTable['rxcui'].map(ab_map))

prescriptionTable

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,rxcui,Antibiotics
0,10418005,22442981,61774509,10418005-61,61.0,P8558I,2130-10-14 17:00:00,2130-10-15 16:00:00,MAIN,Potassium Chloride,...,10mEq/100mL Premix,,40,mEq,400,mL,1.0,IV,312507,False
1,10418005,22442981,61993757,10418005-75,75.0,P8558I,2130-10-14 20:00:00,2130-10-15 19:00:00,MAIN,Heparin,...,5000 Units / mL- 1mL Vial,None,5000,UNIT,1,mL,2.0,SC,1361615,False
2,10418005,22442981,73394102,10418005-37,37.0,P59AIO,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Influenza Vaccine Quadrivalent,...,0.5 mL Syringe,None,0.5,mL,1,SYR,0.0,IM,1801186,False
3,10418005,22442981,77280491,10418005-86,86.0,P8558I,2130-10-15 09:00:00,2130-10-16 18:00:00,MAIN,Ibuprofen,...,600mg Tablet,None,600,mg,1,TAB,NaN,PO,197806,False
4,10418005,22442981,83578254,10418005-89,89.0,P8558I,2130-10-15 11:00:00,2130-10-16 18:00:00,MAIN,Potassium Chloride,...,20mEq Packet,None,60,mEq,3,PKT,1.0,PO,1867544,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15416703,10418005,22442981,54042891,10418005-50,50.0,P59AIO,2130-10-13 20:00:00,2130-10-14 16:00:00,MAIN,Heparin,...,5000 Units / mL- 1mL Vial,None,5000,UNIT,1,mL,2.0,SC,1361615,False
15416704,10418005,22442981,54971858,10418005-48,48.0,P59AIO,2130-10-13 19:00:00,2130-10-15 08:00:00,MAIN,Ketorolac,...,30mg/mL Vial,None,30,mg,1,VIAL,4.0,IV,1665461,False
15416705,10418005,22442981,57377068,10418005-89,89.0,P8558I,2130-10-15 11:00:00,2130-10-16 18:00:00,MAIN,Potassium Chloride,...,10mEq ER Tablet,None,60,mEq,6,TAB,1.0,PO,628958,False
15416706,10418005,22442981,61380456,10418005-64,64.0,P27H2N,2130-10-13 19:00:00,2130-10-16 18:00:00,MAIN,Dextrose 50%,...,25 g / 50 mL Syringe,None,12.5,gm,0.5,SYR,NaN,IV,727517,False


time: 51.1 s (started: 2023-07-20 09:54:17 +08:00)


In [19]:
# this is the end result of code present in: https://github.com/geickelb/Abx_patients_outcome
# As part of a class project, our group performed some review of all antibiotics present in the prescriptionTable with antibiotic==True (based on the ATC code).
# the review involved manual review and regular expressions to identify and filter out any unwanted antibiotics in our dataset. 

#some rows in AB have the same drug name, but differing true/false ab status. this is updating any antibiotic row as true.

#input: imported prescription df annotated with atc codes. 
#ouput: updated prescription df where more true antibiotics are covered

def prescription_update_fxn(prescription_df):
    """
    this function:
    some rows in AB have the same drug name, but differing true/false ab status. this is updating any antibiotic row as true.
    Additionally, there are drugs with antibiotics==True that are not desired for our uses. We want antibiotics that would be prescribed if the physician suspected the patient of having a serious bacterial infection
    
    input: imported prescription df annotated with atc codes. 
    ouput: updated prescription df where more true antibiotics are covered

    """
    drugs_that_dont_belong =['Furosemide','Dextrose 50%','Vancomycin Oral Liquid',
                     'Erythromycin 0.5% Ophth Oint','NEO*IV*Furosemide',
                     'Nystatin','Orthopedic Solution','Neomycin-Polymyxin-Bacitracin Ophth. Oint',
                     'Bacitracin Ophthalmic Oint','Bacitracin Ointment','Lasix',
                     'dextrose','MetronidAZOLE Topical 1 % Gel','Enalaprilat',
                     'NEO*PO*Furosemide (10mg/1ml)','Metronidazole Gel 0.75%-Vaginal','Spironolactone',
                     'Heparin',
                    'voriconazole','valgancyclovir','chloroquine','tamiflu','mefloquine','foscarnet',
                     'fluconazole','vorconazole','quinine','ribavirin','gancyclovir','chloroquine',
                     'atovaquone','ambisome', 'acyclovir', 'Acyclovir']

    drugs_that_dont_belong=[w.lower() for w in drugs_that_dont_belong]
    
    prescriptions= prescription_df
    prescriptions_updated = list(prescriptions.loc[prescriptions.loc[:,"Antibiotics"]==True,'drug'].unique()) 
    true_ndc= prescriptions.loc[prescriptions.loc[:,"drug"].isin(prescriptions_updated),'ndc'].unique() 

    ABrx2= prescriptions.loc[prescriptions.loc[:,"ndc"].isin(true_ndc),:]
    ABrx2= ABrx2.loc[~ABrx2.loc[:,"drug"].str.lower().isin(drugs_that_dont_belong),:] #tilde transforms isin to notin()
    ABrx2['Antibiotics']= True
    return(ABrx2)



time: 0 ns (started: 2023-07-20 09:55:08 +08:00)


In [20]:
#this code takes a while to run, only run if need to update prescription table (which is saved in github)
#note shown: imported antibiotic annotations into prescription table using code provided by
#Dr. Luo's postdoc, which pings API
dtype = {'icustay_id': str,
         'NDC': str,
         'rxcui': str,
         'ingredient': str}
#prescriptions = pd.read_csv('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/Newprescription.csv', index_col=0, dtype=dtype)

#problem: some rows in AB have the same drug name, but differing true/false ab status. this is updating any antibiotic row as true.


ABrx = prescription_update_fxn(prescriptionTable)
ABrx = ABrx.loc[ABrx['Antibiotics'] == True, :]

#save csv:
pd.DataFrame(ABrx).to_csv('c:/Users/csfhe/Documents/RA/mimiciii-antibiotics-opensource/notebooks/data/raw/csv/07192023_ABrx_updated.csv')


time: 15.7 s (started: 2023-07-20 09:55:08 +08:00)


In [21]:
ABrx

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,rxcui,Antibiotics
153,10418018,26872471,12683541,10418018-49,49.0,P48AWH,2160-11-23 16:00:00,2160-11-24 15:00:00,MAIN,CefazoLIN,...,1g Froz.Bag,,1,g,1,BAG,1.0,IV,1665050,True
166,10418018,26872471,73754005,10418018-48,48.0,P48AWH,2160-11-24 16:00:00,2160-11-24 17:00:00,MAIN,Cephalexin,...,500 mg Cap,None,500,mg,1,CAP,3.0,PO/NG,309114,True
202,10418096,23746160,51370446,10418096-54,54.0,P34GXB,2125-12-06 08:00:00,2125-12-07 00:00:00,MAIN,CeFAZolin,...,2 g / 100 mL Dextrose (iso-os),,2,g,1,BAG,3.0,IV,1665060,True
231,10418126,27510878,18984545,10418126-65,65.0,P31MCV,2186-03-16 14:00:00,2186-03-17 13:00:00,MAIN,CeFAZolin,...,1g Froz.Bag,,1,g,1,BAG,3.0,IV,1665050,True
232,10418126,27510878,29737578,10418126-86,86.0,P34GXB,2186-03-19 15:00:00,2186-03-19 20:00:00,MAIN,Amoxicillin,...,250 mg Cap,None,2000,mg,8,CAP,1.0,PO/NG,308182,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15416631,10417827,28142914,55593794,10417827-279,279.0,P16M51,2181-05-08 20:00:00,2181-05-08 14:00:00,MAIN,Vancomycin,...,500mg Vial,,1500,mg,3,VIAL,2.0,IV,1807516,True
15416634,10417827,28142914,59739159,10417827-230,230.0,P12009,2181-05-07 01:00:00,2181-05-08 14:00:00,MAIN,CeftriaXONE,...,1g Frozen Bag,,1,gm,1,BAG,1.0,IV,1665021,True
15416642,10417827,28142914,82237955,10417827-288,288.0,P6802M,2181-05-08 15:00:00,2181-05-12 19:00:00,MAIN,Nafcillin,...,2g Frozen Bag,,2,g,1,BAG,6.0,IV,1721460,True
15416675,10417827,29655156,23628592,10417827-44,44.0,P20P6M,2179-01-03 20:00:00,2179-01-04 09:00:00,MAIN,Vancomycin,...,1g Frozen Bag,,1000,mg,1,BAG,2.0,IV,1807508,True


time: 1.8 s (started: 2023-07-20 09:55:24 +08:00)
